<a href="https://colab.research.google.com/github/DiDarya-art/Collab-Lab/blob/main/Lab%20with%20cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 3
### Выполнил: ФИО

Добро пожаловать в третью лабораторную работу!
Оформите ваше решение контеста Kaggle в соответствии с этим шаблоном. Ваша задача - показать проверяющим ход ваших рассуждений, поэтому советуем писать много комментариев к коду и приводить комментарии к логике на каждом этапе в текстовых ячейках.
Где необходимо, используйте графики для большей наглядности.

При отправке поменяйте название файла на ваши ФИО!

### Настройка Google Colab

Подключение вашего гугл диска (советуем сохранять все важные артефакты именно на диск, чтобы они не удалились при закрытии вкладки с ноутбуком).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Если вы используете google colab и загрузка датасета занимает больше минуты - раскомментируйте и выполните эту ячейку.

In [ ]:
# import json
# import os

# def download_contest3_data():
#   # !mkdir ~/.kaggle # закоментить если ругается
#   # !touch ~/.kaggle/kaggle.json # закоментить если ругается

#   # токен надо сгенерировать в личном кабинете на kaggle (https://www.kaggle.com/settings/account)
#   api_token = {"username":"fokuspokus","key":"abrakadabra"}
#   with open('/root/.kaggle/kaggle.json', 'w') as file:
#       json.dump(api_token, file)

#   !chmod 600 ~/.kaggle/kaggle.json

#   !kaggle competitions download -p /content/drive/MyDrive/ -c ml-mipt-2023-contest-3

#   if not os.path.isdir("/content/drive/MyDrive/contest3"):
#     !mkdir /content/drive/MyDrive/contest3

#   !unzip /content/drive/MyDrive/ml-mipt-2023-contest-3.zip -d /content/drive/MyDrive/contest3

# download_contest3_data()

In [ ]:
!cp -r /content/drive/MyDrive/contest3 /content/contest3

### EDA (исследовательский анализ данных)

В этой лабораторной вы работаете с картинками. Напишите функцию, которая отрисовывала бы ключевые точки на конкретном изображении, это вам пригодится для дебага.
Можете посчитать статистики по картинкам в датасете.

Подключение библиотек:

In [ ]:
import numpy as np
import os
import cv2
import random
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

Фиксирование seed:

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Рассмотрим данные:

In [ ]:
TRAIN_PATH = "contest3/train_labels.csv"
IMAGE_TRAIN_PATH = "contest3/images/train"
IMAGE_TEST_PATH = "contest3/images/test"
TARGET_SIZE = 224
train_data = pd.read_csv(TRAIN_PATH)
train_data.head()

In [ ]:
train_data.info()

In [ ]:
keypoints = train_data.columns[:-1]
keypoints

Подберём разные цвета для точек:

In [ ]:
DEFAULT_POINT_COLORS = [
    (255, 0, 0), (150, 0, 0), (255, 0, 255),
    (150, 255, 0), (0, 255, 0), (0, 150, 0),
    (0, 255, 255), (0, 150, 255), (0, 0, 255)
]

Функция отрисовки:

In [ ]:
def draw_keypoints_on_image(img, keypoints, radius=5, point_colors=DEFAULT_POINT_COLORS):
    """
    Рисует ключевые точки на уже загруженном RGB изображении (img)
    Возвращает изменённое изображение
    """
    img_copy = img.copy()
    for i in range(0, len(keypoints), 2):
        x = int(keypoints[i])
        y = int(keypoints[i + 1])
        color = point_colors[i//2]
        cv2.circle(img_copy, (x, y), radius, color, -1)
    return img_copy

In [ ]:
def draw_keypoints(image_path, keypoints, ax, radius=5, point_colors=DEFAULT_POINT_COLORS):
    """
    image_path: путь к картинке
    keypoints: массив ключевых точек (18,) [x1, y1, x2, y2, ...]
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = draw_keypoints_on_image(img, keypoints, radius, point_colors)
    ax.imshow(img)
    ax.axis("off")

Пример изображений:

In [ ]:
sample_rows = train_data.sample(6, random_state=seed)
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for ax, (_, row) in zip(axes.flatten(), sample_rows.iterrows()):
    file_name = row["file_name"]
    image_path = os.path.join(IMAGE_TRAIN_PATH, file_name)
    kp = row[keypoints].to_numpy()
    draw_keypoints(image_path, kp, ax)
    ax.set_title(file_name)
plt.tight_layout()
plt.show()

Количество изображений и количество полей различны, проверим уникальность file_name:

In [ ]:
train_data['file_name'].is_unique

Выведем строки с неуникальными значениями filename:

In [ ]:
duplicates = train_data[train_data.duplicated(subset='file_name', keep=False)]
duplicates

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(6, 6))
for ax, (_, row) in zip(axes.flatten(), duplicates.iterrows()):
    file_name = row["file_name"]
    image_path = os.path.join(IMAGE_TRAIN_PATH, file_name)
    kp = row[keypoints].to_numpy()
    draw_keypoints(image_path, kp, ax)
    ax.set_title(f"Index: {row.name}")
plt.tight_layout()
plt.show()

In [ ]:
train_data = train_data.drop(index=[140, 448]).reset_index(drop=True)
train_data['file_name'].is_unique

Статистика датасета:

In [ ]:
train_data[keypoints].describe()

Анализ размеров изображений:

In [ ]:
heights = []
widths = []
for filename in train_data["file_name"]:
    img = cv2.imread(os.path.join(IMAGE_TRAIN_PATH, filename))
    h, w, _ = img.shape
    heights.append(h)
    widths.append(w)

print("Height:\nmin:", min(heights), " max:", max(heights))
print("Width:\nmin:", min(widths), " max:", max(widths))

In [ ]:
plt.figure(figsize=(7, 3))
plt.subplot(1, 2, 1)
plt.scatter(widths, heights, alpha=0.5)
plt.xlabel("Width")
plt.ylabel("Height")
plt.title("Разброс размеров картинок")
plt.show()

Точки за пределами изображений:

In [ ]:
images = []
for idx, row in train_data.iterrows():
    filename = row["file_name"]
    image_path = os.path.join(IMAGE_TRAIN_PATH, filename)
    h, w, _ = cv2.imread(image_path).shape
    out_of_bounds = False
    for i in range(0, len(keypoints), 2):
        x = row[keypoints[i]]
        y = row[keypoints[i + 1]]
        if x < 0 or y < 0 or x >= w or y >= h:
            out_of_bounds = True
            break

    if out_of_bounds:
        images.append(filename)

print(f"Всего картинок с точками за пределами изображения: {len(images)}")

Выводы:
- Некоторые точки выходят за границу изображения.
- Изображения имеют разный размер. Прямой resize к квадратному формату может искажать пропорции, поэтому на этапе препроцессинга необходимо использовать padding с сохранением соотношений сторон.
- Большой разброс котов.

### Preprocessing (подготовка данных)

Мы ожидаем, что в этом разделе вы реализуете кастомный класс для подгрузки данных (cоветуем унаследоваться от `torch.utils.data.Dataset`).

In [ ]:
# КОСЯКИ.  сделал 2 варианта с препроцессом и без, препроцесс вроде ускорил

In [ ]:
def prepare_images(df, src_dir, out_dir, keypoints_cols, target_size=224):
    os.makedirs(out_dir, exist_ok=True)
    is_train = df is not None

    if is_train:
        new_df = df.copy()
        filenames = new_df["file_name"].values
    else:
        filenames = sorted([f for f in os.listdir(src_dir) if f.endswith('.jpg')])

    for filename in tqdm(filenames):
        img_path = os.path.join(src_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Propblem with {filename}")
            continue

        h, w = img.shape[:2]
        scale = target_size / max(h, w)
        nw, nh = int(w * scale), int(h * scale)

        # Ресайз и паддинг
        img_res = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
        canvas = np.zeros((target_size, target_size, 3), dtype=np.uint8)
        pad_x, pad_y = (target_size - nw) // 2, (target_size - nh) // 2
        canvas[pad_y:pad_y+nh, pad_x:pad_x+nw] = img_res

        cv2.imwrite(os.path.join(out_dir, filename), canvas)

        if is_train:
            idx = new_df[new_df["file_name"] == filename].index[0]
            pts = new_df.loc[idx, keypoints_cols].values.astype(np.float32)
            pts[0::2] = (pts[0::2] * scale + pad_x) / target_size
            pts[1::2] = (pts[1::2] * scale + pad_y) / target_size
            new_df.loc[idx, keypoints_cols] = pts

    return new_df if is_train else None

In [ ]:
train_data224 = prepare_images(train_data, IMAGE_TRAIN_PATH, "contest3/train224", keypoints)
prepare_images(None, IMAGE_TEST_PATH, "contest3/test224", keypoints)

In [ ]:
class CatDataset(Dataset):
    def __init__(self, image_dir, data=None, transform=None):
        """
        image_dir: путь к папке с картинками 224x224
        data: DataFrame, точки под 224x224
        """
        self.image_dir = image_dir
        self.transform = transform
        self.is_train = data is not None
        if self.is_train:
            self.filenames = data["file_name"].values
            self.points = data.drop(columns=["file_name"]).values.astype(np.float32)
        else:
            self.filenames = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])

    def __len__(self) -> int:
        return len(self.filenames)

    def __getitem__(self, idx: int):
        file_name = self.filenames[idx]
        img_path = os.path.join(self.image_dir, file_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.transpose(2, 0, 1).astype(np.float32) / 255.0
        img = torch.from_numpy(img)

        if self.transform:
            img = self.transform(img)

        if not self.is_train:
            return img, file_name
        return img, torch.from_numpy(self.points[idx])

In [ ]:
train_df, val_df = train_test_split(
    train_data224,
    test_size=0.2,
    random_state=seed
)
datasets = {
    'train': CatDataset("contest3/train224", data=train_df),
    'val': CatDataset("contest3/train224", data=val_df),
    'test': CatDataset("contest3/test224")
}

Проверим работу датасета:

In [ ]:
img, points = datasets['train'][0]
img_np = img.permute(1, 2, 0).numpy()
points = points.numpy() * TARGET_SIZE

plt.figure(figsize=(5,5))
plt.imshow(img_np)
for i in range(0, len(points), 2):
    plt.scatter(points[i], points[i+1], c='r', s=10)
plt.title("Dataset image")
plt.axis("off")
plt.show()


### Training/evaluation loop

В этом разделе напишите функцию, принимающую модель, оптимизатор, кол-во эпох, и т.д, которая осуществляет обучение с заданными параметрами. Подумайте, что функция будет возвращать.

Смысл этого раздела в том, чтобы не дублировать код обучения для каждого эксперимента. А еще на такую функцию легко накинуть перебор гиперпараметров...

In [ ]:
def train_model(model, dataloaders, optimizer, num_epochs,
                criterion, device, patience_early_stop=5, path="best_model.pth"):
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_mae': [],
        'val_mae': []
    }
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
        mode='min', factor=0.5, patience=5)
    best_val_loss = float('inf')
    early_stop_counter = 0
    model.to(device)
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # train
            else:
                model.eval()  # validation
            run_loss = 0.0
            run_mae = 0.0
            # Итерация по батчам
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                # Вычисление градиентов происходит только при обучении
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    mae = torch.abs(outputs - labels).mean()
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                run_loss += loss.item() * inputs.size(0)
                run_mae += mae.item() * inputs.size(0)
            # Вычисление средней ошибки
            run_loss = run_loss / len(dataloaders[phase].dataset)
            run_mae = run_mae / len(dataloaders[phase].dataset)
            history[f'{phase}_loss'].append(run_loss)
            history[f'{phase}_mae'].append(run_mae)

            if phase == 'val':
                # ReduceLROnPlateau
                scheduler.step(run_loss)
                # Сохранение лучшей модели
                if run_loss < best_val_loss:
                    best_val_loss = run_loss
                    torch.save(model.state_dict(), path)
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1
        print(f"Epoch {epoch+1}: Train Loss: {history['train_loss'][-1]:.4f} | Val Loss: {history['val_loss'][-1]:.4f}")
        # Early stopping
        if early_stop_counter >= patience_early_stop:
            break
    return history

### Prediction function

Реализуйте функцию, которая бы делала предсказания. Функция принимает датасет/даталоадер и модель (мб еще что-то). Эта функция нужна вам, чтобы было удобнее считать метрику (по сути она будет склеивать предсказания из батчей в один массив).

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    model.to(device)
    predictions = []
    targets = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())
            targets.append(labels.cpu().numpy())
    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    return predictions, targets

### Experiments

В этом разделе задаете архитектуры моделей и ставите эксперименты по обучению. Мы ожидаем, что вы зададите модель в виде класса (унаследуетесь от класса `torch.nn.Module`).

Если вы ставили много экспериментов, приведите их в хронологическом порядке, чтобы мы увидели эволюцию ваших рассуждений.

Мы ожидаем увидеть графики train/val лоссов и метрик для ваших экспериментов. Мы ожидаем увидеть визуализацию примеров предсказаний.

Функция для построения графиков:

In [ ]:
def draw_plots(history):
    """
    Строит графики обучения модели
    """
    _, axes = plt.subplots(1, 2, figsize=(12, 4))

    # График 1: Loss
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_title('Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)

    # График 2: Columnwise MAE
    axes[1].plot(history['train_mae'], label='Train MAE')
    axes[1].plot(history['val_mae'], label='Val MAE')
    axes[1].set_title('Columnwise MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_predictions(image_path, kp, pred_kp, ax, color=DEFAULT_POINT_COLORS):
    darkened_color = []
    for c in color:
        new_c = tuple(int(max(0, min(255, x * 0.5))) for x in c)
        darkened_color.append(new_c)
    img = draw_keypoints(image_path, kp, ax, radius=5)
    img = draw_keypoints_on_image(img, pred_kp, radius=3, point_colors=darkened_color)
    ax.imshow(img)
    ax.axis("off")

def compute_mae_px(preds, targets, image_size=224):
    """
    preds, targets: нормализованные [0,1]
    Возвращает среднюю ошибку в пикселях
    """
    preds_px = preds * image_size
    targets_px = targets * image_size
    mae_px = np.abs(preds_px - targets_px).mean()
    return mae_px

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
BATCH_SIZE = 64
NUM_WORKERS = 2
generator = torch.Generator()
generator.manual_seed(seed)
dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, pin_memory=True,
                        generator=generator),
    'val': DataLoader(datasets['val'], batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=True),
    'test': DataLoader(datasets['test'], batch_size=BATCH_SIZE, shuffle=False,
                       num_workers=NUM_WORKERS, pin_memory=True)
}

#### Эксперимент 1

Создадим простую сверточную нейронную сеть:

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_keypoints=18):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7))
        )

        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_keypoints)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.regressor(x)
        return x

Визуализация архитектуры:

In [ ]:
from torchsummary import summary

model1 = SimpleCNN().to(device)
summary(model1, input_size=(3, 224, 224))

In [ ]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
num_epochs = 50

In [ ]:
xb, yb = next(iter(dataloaders['train']))
print(xb.shape)  # [B, 3, H, W]
print(yb.shape)  # [B, num_keypoints]


In [ ]:
model1.eval()
with torch.no_grad():
    out = model1(xb.to(device))
print(out.shape)


In [ ]:
history1 = train_model(model1, dataloaders, optimizer, num_epochs, criterion, device)

In [ ]:
model1.load_state_dict(torch.load("best_model.pth"))

#### Эксперимент 2

In [ ]:
class IntermediateCNN(nn.Module):
    def __init__(self, num_keypoints=18):
        super().__init__()
        # Вход 224x224
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2), # 112x112

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2), # 56x56

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2), # 28x28

            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d(2), # 14x14

            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7)) # Фиксируем размер перед FC слоем
        )

        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_keypoints)
        )

    def forward(self, x):
        return self.regressor(self.features(x))

In [ ]:
model2 = IntermediateCNN().to(device)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
num_epochs = 50

In [ ]:
%%time
history2 = train_model(model2, dataloaders, optimizer, num_epochs, criterion, device, patience_early_stop=7, path="best_model2_10.pth")

In [ ]:
draw_plots(history2)

In [ ]:
class SmallCNN(nn.Module):
    def __init__(self, num_keypoints=18):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),  # 112x112

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),  # 56x56

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),  # 28x28

            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7))
        )

        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*7*7, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_keypoints)
        )

    def forward(self, x):
        return self.regressor(self.features(x))


In [ ]:
model24 = SmallCNN().to(device)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model24.parameters(), lr=1e-3)
num_epochs = 50

In [ ]:
%%time
history24 = train_model(model24, dataloaders, optimizer, num_epochs, criterion, device, patience_early_stop=7, path="best_model2_4.pth")

In [ ]:
draw_plots(history2)

In [ ]:
class SmallCNN2(nn.Module):
    def __init__(self, num_keypoints=18):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),  # 112x112

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),  # 56x56

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),  # 28x28

            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7))
        )

        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*7*7, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_keypoints)
        )

    def forward(self, x):
        return self.regressor(self.features(x))


In [ ]:
model2d = SmallCNN2().to(device)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model2d.parameters(), lr=1e-3)
num_epochs = 50

In [ ]:
%%time
history24 = train_model(model2d, dataloaders, optimizer, num_epochs, criterion, device, patience_early_stop=7, path="best_model2_d.pth")

In [ ]:
draw_plots(history2)

#### Эксперимент 3

In [ ]:
class ResCNN(nn.Module):
    def __init__(self, num_keypoints: int = 18, base: int = 32, drop2d: float = 0.02, dropout: float = 0.2):
        super().__init__()
        self.base = base

        # Stem: 5 каналов -> 32
        self.stem = nn.Sequential(
            nn.Conv2d(5, base, 3, padding=1, bias=False),
            nn.BatchNorm2d(base),
            nn.SiLU(inplace=True),
        )

        # Блоков делаем поровну: [2, 2, 2, 2]
        self.stage1 = nn.Sequential(
            ResBlock(base, base, stride=1, drop2d=drop2d),
            ResBlock(base, base, stride=1, drop2d=drop2d),
        )
        self.stage2 = nn.Sequential(
            ResBlock(base, base*2, stride=2, drop2d=drop2d),
            ResBlock(base*2, base*2, stride=1, drop2d=drop2d),
        )
        self.stage3 = nn.Sequential(
            ResBlock(base*2, base*4, stride=2, drop2d=drop2d),
            ResBlock(base*4, base*4, stride=1, drop2d=drop2d), # Убрали лишний блок
        )
        self.stage4 = nn.Sequential(
            ResBlock(base*4, base*8, stride=2, drop2d=drop2d),
            ResBlock(base*8, base*8, stride=1, drop2d=drop2d),
        )

        # Сжатие: base*8 (256 каналов при base=32)
        self.compress = nn.Sequential(
            nn.Conv2d(base*8, base*4, kernel_size=1, bias=False),
            nn.BatchNorm2d(base*4),
            nn.SiLU(inplace=True),
        )

        self.pool = nn.AdaptiveAvgPool2d((4, 4))

        # Упрощенная голова: меньше слоев = меньше оверфита
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(base*4*4*4, 512), # 128*4*4 = 2048 -> 512
            nn.SiLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_keypoints) # Сразу выход
        )

    def _add_coord_channels(self, x):
        b, _, h, w = x.shape
        ys = torch.linspace(-1, 1, steps=h, device=x.device).view(1, 1, h, 1).expand(b, 1, h, w)
        xs = torch.linspace(-1, 1, steps=w, device=x.device).view(1, 1, 1, w).expand(b, 1, h, w)
        return torch.cat([x, xs, ys], dim=1)

    def forward(self, x):
        x = self._add_coord_channels(x)
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.compress(x)
        x = self.pool(x)
        return self.head(x)

In [ ]:
model3 = ResCNN().to(device)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model2d.parameters(), lr=1e-3)
num_epochs = 50

In [ ]:
def postprocess_preds(preds, original_widths, original_heights, target_size=224):
    """
    preds: тензор (N, 18) в диапазоне [0, 1]
    returns: тензор с реальными пикселями
    """
    rescaled_preds = preds.copy()
    for i in range(len(preds)):
        w, h = original_widths[i], original_heights[i]
        scale = target_size / max(h, w)
        nw, nh = int(w * scale), int(h * scale)
        pad_x, pad_y = (target_size - nw) // 2, (target_size - nh) // 2

        # Сначала из [0, 1] в [0, 224]
        rescaled_preds[i] *= target_size
        # Вычитаем паддинг и делим на коэффициент масштаба
        rescaled_preds[i, 0::2] = (rescaled_preds[i, 0::2] - pad_x) / scale
        rescaled_preds[i, 1::2] = (rescaled_preds[i, 1::2] - pad_y) / scale

    return rescaled_preds

### Evaluation  (оценка качества модели)

В этом разделе проводите оценку качества вашей итоговой модели (с помощью prediction function из раздела выше).

добавить преобразования для теста

In [ ]:
# model evaluation

### Conclusion (Выводы)

В этом разделе описываете полученные результаты и проводите анализ выполненной работы.
Что получилось / не получилось и почему?